# Purchase Card Transactions

This notebook is responsible for the analysis of purchase card transactions for the first half of 2022.

Dataset: https://www.cityobservatory.birmingham.gov.uk/@birmingham-city-council/purchase-card-transactions

In [ ]:
!pip install ydata-profiling

In [3]:
import joblib
import random

import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
import plotly.graph_objects as go

## Data collection and analysis

 First of all, let's download the database. Since we're going to analyze the transactions for the first half of 2022, and since the database is divided by months, we need to download six ***xls*** files.


For each of the files we're going to create a Dataframe and finally we're going to join them based on the columns they have in common.

In [4]:
transactions_january_path = "https://birmingham-city-observatory.datopian.com/dataset/cf552d08-cee9-43bf-8c0f-3196a9311799/resource/b4fa2e44-62b6-43b9-9a4d-cfb7acb18adb/download/purchase-card-transactions-january-2022.xls"
transactions_february_path = "https://birmingham-city-observatory.datopian.com/dataset/cf552d08-cee9-43bf-8c0f-3196a9311799/resource/0f808dbe-727a-446b-bd08-cf08b29fe67a/download/purchase-card-transactions-february-2022.xls"
transactions_march_path = "https://birmingham-city-observatory.datopian.com/dataset/cf552d08-cee9-43bf-8c0f-3196a9311799/resource/6408d4ad-4f9d-4751-ad38-b9306c1629a6/download/purchase-card-transactions-march-2022.xls"
transactions_april_path = "https://birmingham-city-observatory.datopian.com/dataset/cf552d08-cee9-43bf-8c0f-3196a9311799/resource/3893f300-7135-4928-9e74-2e4127002fa3/download/purchase-card-transactions-april-2022.xls"
transactions_may_path = "https://birmingham-city-observatory.datopian.com/dataset/cf552d08-cee9-43bf-8c0f-3196a9311799/resource/18214ca4-a491-4220-a592-1747aad401c5/download/purchase-card-transactions-may-2022.xls"
transactions_june_path = "https://birmingham-city-observatory.datopian.com/dataset/cf552d08-cee9-43bf-8c0f-3196a9311799/resource/abec4185-1fc7-4bac-a159-c77a88f0d204/download/purchase-card-transactions-june-2022.xls"

In [5]:
transactions_january = pd.read_excel(transactions_january_path)
transactions_february = pd.read_excel(transactions_february_path)
transactions_march = pd.read_excel(transactions_march_path)
transactions_april = pd.read_excel(transactions_april_path)
transactions_may = pd.read_excel(transactions_may_path)
transactions_june = pd.read_excel(transactions_june_path)

In [6]:
def standardize_columns(dataframe):
    """
    Standardizes the names of the dataframe columns.

    Args:
        * dataframe (pd.DataFrame): DataFrame whose columns are to be standardized.

    Returns:
        * (pd.DataFrame): DataFrame with standardized column names.
    """
    # Convert all column names to uppercase
    dataframe.columns = [col.upper() for col in dataframe.columns]
    return dataframe

def merge_dataframes(*dfs):
    """
    Join multiple dataframes based on common columns.

    Args:
        * dfs (list): Dataframes to join.

    Returns:
        * (pd.Dataframe): Joined DataFrame.
    """
    common_columns = [
        "TRANS DATE", "TRANS TAX DESC", "ORIGINAL GROSS AMT", "ORIGINAL CUR",
        "BILLING CUR CODE", "BILLING GROSS AMT", "MERCHANT NAME", "CARD NUMBER",
        "TRANS CAC CODE 1", "TRANS CAC CODE 2", "DIRECTORATE",
    ]

    # Standardize column names and join dataframes using only the columns in common
    merged_df = pd.concat([standardize_columns(df)[common_columns] for df in dfs], ignore_index=True)

    return merged_df

In [7]:
transactions_data = merge_dataframes(transactions_january, transactions_february, transactions_march, transactions_april, transactions_may, transactions_june)

Using the ProfileReport method (from the ydata_profiling library) we can obtain a statistical report of the entire dataset we have organized.

In [8]:
profile = ProfileReport(transactions_data, title="Profiling Report")
profile

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Based on the results of this report, we see that the variables "ORIGINAL GROSS AMT" and "BILLING GROSS AMT" are highly correlated. Reviewing the dataframe we also see that in general both columns have the same values, so it is best to work only with one of the two columns and not with both.

On the other hand we see that there is also correlation between "ORIGINAL CUR" and "ORIGINAL GROSS AMT". This correlation is a bit more difficult to interpret without additional context, since "ORIGINAL CUR" appears to be a categorical variable (indicating currency). However, if it has been coded in some way and a correlation has been found, it could indicate a relationship between the currency used and the original gross amount.

We also see that there are several duplicate records.

The variables "DIRECTORATE", "TRANS TAX DESC", "ORIGINAL CUR" have unbalanced data, meaning that some categories are overrepresented compared to others. There are several techniques that can be used to deal with unbalanced data, especially when dealing with categorical characteristics. The technique we're going to implement is Frequency Coding. So, instead of using ordinal or one-hot coding, we're going to code categories based on the frequency (or probability) of each category in the data set. More frequent categories will have higher values.

We'll proceed to deal with all these cases.

In [10]:
class DataProcessor:
    """
    Data Processor class to handle preprocessing of transaction data.
    """

    def __init__(self, transactions_data):
        """
        Constructor method.

        Args:
            * transactions_data (pd.DataFrame): Raw transaction data.
        """
        self.data = transactions_data.copy()

    def extract_date_features(self):
        """
        Extract features from the 'TRANS DATE' column.
        """
        print("Extract date features")
        self.data['TRANS DATE'] = pd.to_datetime(self.data['TRANS DATE'])
        self.data['Year'] = self.data['TRANS DATE'].dt.year
        self.data['Month'] = self.data['TRANS DATE'].dt.month
        self.data['Day'] = self.data['TRANS DATE'].dt.day
        self.data['DayOfWeek'] = self.data['TRANS DATE'].dt.dayofweek

    def handle_missing_values(self):
        """
        Handle missing values in the data.
        """
        print("Handle missing values")
        # Replace missing values with the mean of the column
        for column in self.data.columns:
            if self.data[column].isnull().any():
                # Check if the column is numeric
                if self.data[column].dtype in ['int64', 'float64']:
                    self.data[column].fillna(self.data[column].mean(), inplace=True)
                else:
                    # Replace with the most frequent value for non-numeric columns
                    self.data[column].fillna(self.data[column].mode()[0], inplace=True)

    def normalize_columns(self, column_to_normalize):
        """
        Normalize certain columns for better model performance.
        """
        print(f"Normalize '{column_to_normalize}' column")
        self.data[column_to_normalize] = (self.data[column_to_normalize] -
                self.data[column_to_normalize].min()) / (self.data[column_to_normalize].max() -
                    self.data[column_to_normalize].min())

    def encode_categorical_features(self, categorical_columns):
        """
        Encode categorical features.

        Args:
            * categorical_columns (list): Categorical columns.
        """
        print("Encode categorical features using Label Encoding")
        # Apply Label Encoding for each categorical column
        for column in categorical_columns:
            # Convert to uppercase to standardize the values
            self.data[column] = self.data[column].str.upper()

            # Initialize the LabelEncoder
            le = LabelEncoder()

            # Fit and transform the data
            self.data[column] = le.fit_transform(self.data[column])

    def group_less_frequent_categories(self, columns, threshold=0.045):
        """
        Group less frequent categories into a new category called 'Others'.

        Args:
            columns (list): List of columns to apply the grouping.
            threshold (float): The frequency threshold below which categories will be grouped into 'Others'.
                            Default is 0.045 (4.5%).

        Returns:
            pd.DataFrame: Dataframe with less frequent categories grouped.
        """
        for column in columns:
            print(f"Group less frequent categories into a new category called 'Others' for {column}")
            # Calculate the frequency of each category
            freq = self.data[column].value_counts(normalize=True)

            # Identify categories with frequency below the threshold
            mask = self.data[column].isin(freq[freq < threshold].index)

            # Replace those categories with 'Others'
            self.data.loc[mask, column] = 'Others'


    def drop_unnecessary_columns(self, columns_to_drop):
        """
        Drop columns that are not necessary for forecasting.

        Args:
            * columns_to_drop (list): Columns to drop.
        """
        print("Drop columns")
        # List of columns to drop based on the analysis
        self.data.drop(columns=columns_to_drop, inplace=True, errors='ignore')

    def remove_duplicates(self):
        """
        Remove duplicate rows from the data.
        """
        print("Remove duplicate rows")
        self.data.drop_duplicates(inplace=True)

    def get_processed_data(
        self,
        column_to_normalize,
        categorical_columns,
        immbalance_colummns,
        columns_to_drop,
    ):
        """
        Return the cleaned and processed data.

        Args:
            * column_to_normalize (list): Column to normalize.
            * categorical_columns (list): Categorical columns.
            * immbalance_colummns (list): Categorical columns (imbalance).
            * columns_to_drop (list): Columns to drop.

        Returns:
            * (pd.DataFrame): Processed data.
        """
        self.extract_date_features()
        self.handle_missing_values()
        self.normalize_columns(column_to_normalize)
        self.encode_categorical_features(categorical_columns)
        self.group_less_frequent_categories(immbalance_colummns)
        self.encode_categorical_features(immbalance_colummns)
        self.drop_unnecessary_columns(columns_to_drop)
        self.remove_duplicates()
        return self.data

In [11]:
data_processor = DataProcessor(transactions_data)

column_to_normalize = "ORIGINAL GROSS AMT"
categorical_columns = ["MERCHANT NAME"]
immbalance_colummns = ["TRANS TAX DESC", "ORIGINAL CUR", "DIRECTORATE"]

columns_to_drop = ["TRANS DATE", "BILLING CUR CODE", "BILLING GROSS AMT", "CARD NUMBER", "TRANS CAC CODE 1", "TRANS CAC CODE 2"]

processed_data = data_processor.get_processed_data(
    column_to_normalize,
    categorical_columns,
    immbalance_colummns,
    columns_to_drop,
)

Extract date features
Handle missing values
Normalize 'ORIGINAL GROSS AMT' column
Encode categorical features using Label Encoding
Group less frequent categories into a new category called 'Others' for TRANS TAX DESC
Group less frequent categories into a new category called 'Others' for ORIGINAL CUR
Group less frequent categories into a new category called 'Others' for DIRECTORATE
Encode categorical features using Label Encoding
Drop columns
Remove duplicate rows


In [12]:
processed_data.head()

,TRANS TAX DESC,ORIGINAL GROSS AMT,ORIGINAL CUR,MERCHANT NAME,DIRECTORATE,Year,Month,Day,DayOfWeek
0,1,0.008798,0,2165,1,2022,1,14,4
1,1,0.008821,0,2165,1,2022,1,19,2
2,1,0.008845,0,2165,1,2022,1,19,2
3,1,0.008791,0,2165,1,2022,1,26,2
4,1,0.008812,0,2165,1,2022,1,27,3


## Train model

Since the objective is to predict future transactional behaviors, such as next purchase or spending forecast, it is essential to consider all relevant characteristics that could influence these behaviors. In this case, the characteristics to be used for ***X*** (independent variables) will be all columns except 'ORIGINAL GROSS AMT', which will be our target variable ***y***.

To validate the model we're using the RMSE (Root Mean Squared Error) and the MAE (Mean Absolute Error), which are common metrics to evaluate the accuracy of regression models. Both metrics indicate the error between model predictions and actual values, but interpret this error in different ways.

* RMSE (Root Mean Squared Error):
    Gives an idea of how much error there is between two data sets. In other words, it compares a predicted value and an observed or known value.
    It penalizes large errors more than the MAE. That is, if there are large deviations between predictions and actual values, the RMSE will be significantly higher than the MAE.
    A lower RMSE value is better than a higher value, since it indicates that the model predictions are closer to the actual values.

* MAE (Mean Absolute Error):
    This is the average of the absolute errors between the predictions and the actual values.
    It provides an idea of the magnitude of the error, but not its direction (i.e., it does not indicate whether the predictions are above or below the actual values).
    Like RMSE, a lower MAE value is better.

In [13]:
class ForecastModel:
    """
    Forecast Model
    """

    def __init__(self, data):
        """
        Constructor method

        Args:
            * data (pd.DataFrame): Dataframe containing the transaction data.
        """
        self.data = data
        self.model = None
        self.X_train, self.X_test, self.y_train, self.y_test = None, None, None, None

    def train_model(self):
        """
        Train the forecasting model using Random Forest Regressor.
        """
        # Features and target variable
        X = self.data.drop(columns=['ORIGINAL GROSS AMT'])
        y = self.data['ORIGINAL GROSS AMT']

        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.2)

        self.model = RandomForestRegressor()
        self.model.fit(self.X_train, self.y_train)

    def validate_model(self):
        """
        Validate the trained model using the test set and return error metrics.

        Returns:
            * (dict): Dictionary containing RMSE and MAE.
        """
        predictions = self.model.predict(self.X_test)
        rmse = np.sqrt(mean_squared_error(self.y_test, predictions))
        mae = mean_absolute_error(self.y_test, predictions)
        return {"RMSE": rmse, "MAE": mae}

    def predict(self, input_data):
        """
        Predict the 'ORIGINAL GROSS AMT' for given input data.

        Args:
            * input_data (pd.DataFrame): Dataframe containing the features for prediction.

        Returns:
            * (np.array): Predicted 'ORIGINAL GROSS AMT' values.
        """
        return self.model.predict(input_data)

    def save_model(self, filename):
        """
        Save the trained model to a file.

        Args:
            * filename (str): Name of the file to save the model.
        """
        joblib.dump(self.model, filename)

    def load_model(self, filename):
        """
        Load a trained model from a file.

        Args:
            * filename (str): Name of the file from which to load the model.
        """
        self.model = joblib.load(filename)


In [14]:
forecast_model = ForecastModel(processed_data)
forecast_model.train_model()

# Validar el modelo y mostrar métricas de error
validation_metrics = forecast_model.validate_model()
print(f"RMSE: {validation_metrics['RMSE']}")
print(f"MAE: {validation_metrics['MAE']}")

RMSE: 0.010926787645005777
MAE: 0.0006988626097252323


Since the RMSE and MAE values are quite low, this suggests that the model performs well and the predictions are close to the true values.

## Visualize results

In [15]:
class ResultsVisualizer:
    """
    Results Visualizer
    """

    def __init__(self, model, feature_names):
        """
        Constructor method

        Args:
            * model (object): Trained forecasting model.
            * feature_names (list): List of feature names.
        """
        self.model = model
        self.feature_names = feature_names

    def plot_results(self, X_test, y_test):
        """
        Plot the real vs predicted values.

        Args:
            * X_test (np.array): Test data.
            * y_test (np.array): True values for the test data.
        """
        predictions = self.model.predict(X_test)

        fig = go.Figure()
        fig.add_trace(go.Scatter(y=y_test, mode='lines', name='Real Data'))
        fig.add_trace(go.Scatter(y=predictions, mode='lines', name='Predicted Data'))
        fig.update_layout(title="Real vs Predicted Values", xaxis_title="Data Points", yaxis_title="Values")
        fig.show()

    def plot_residuals(self, X_test, y_test):
        """
        Plot the residuals (difference between real and predicted values).

        Args:
            * X_test (np.array): Test data.
            * y_test (np.array): True values for the test data.
        """
        predictions = self.model.predict(X_test)
        residuals = y_test - predictions

        fig = go.Figure()
        fig.add_trace(go.Scatter(x=y_test, y=residuals, mode='markers'))
        fig.update_layout(title="Residuals Plot", xaxis_title="Real Values", yaxis_title="Residuals")
        fig.show()


In [16]:
# Visualizar los resultados reales vs predichos
feature_names = ['TRANS TAX DESC', 'ORIGINAL GROSS AMT', 'ORIGINAL CUR', 'MERCHANT NAME', 'DIRECTORATE', 'Year', 'Month', 'Day', 'DayOfWeek']
visualizer = ResultsVisualizer(forecast_model, feature_names)

X_test = forecast_model.X_test  # Usamos los datos de prueba que se separaron durante el entrenamiento
y_test = forecast_model.y_test  # Usamos los valores reales correspondientes a X_test
visualizer.plot_results(X_test, y_test)
visualizer.plot_residuals(X_test, y_test)

* Actual vs Predicted Values:      
    This plot compares the actual (observed) values to the values predicted by the model.
    Deviations between actual and predicted values indicate errors in the predictions.
    This visualization helps to identify areas where the model may be underestimating or overestimating values.

* Residuals Plot:     
    This plot shows the difference between the actual and predicted values, known as residuals.
    Ideal residuals would be randomly distributed around zero, with no discernible patterns.

## Predictions

1. Forecasting future transactional behavior

To guess future transactional behaviors, we trained a regression model that uses the date to predict the transaction amount (BILLING GROSS AMT).
What would be the next purchase?

To predict the amount of the next purchase, we can enter some date this year and get a prediction. For example:

In [17]:
future_data = pd.DataFrame({
    'TRANS DATE': ["2023-06-21"],
    'TRANS TAX DESC': ["VZ"],
    'ORIGINAL GROSS AMT': [204.0],
    'ORIGINAL CUR': ["GBP"],
    'BILLING CUR CODE': ["GBP"],
    'BILLING GROSS AMT': [204.0],
    'MERCHANT NAME': ["amznmktplace"],
    'CARD NUMBER': ["*****1234"],
    'TRANS CAC CODE 1': ["L100"],
    'TRANS CAC CODE 2': ["REAAB"],
    'DIRECTORATE': ["NEIGHBOURHOODS"],
})

# Usar DataProcessor para procesar los datos futuros
data_processor = DataProcessor(future_data)
processed_future_data = data_processor.get_processed_data(
    column_to_normalize="ORIGINAL GROSS AMT",
    categorical_columns=["MERCHANT NAME"],
    immbalance_colummns=["TRANS TAX DESC", "ORIGINAL CUR", "DIRECTORATE"],
    columns_to_drop=["TRANS DATE", "BILLING CUR CODE", "BILLING GROSS AMT", "CARD NUMBER", "TRANS CAC CODE 1", "TRANS CAC CODE 2"]
)

# Let's make sure that the processed_future_data columns match the training columns.
# This is crucial because the model expects to receive the same features with which it was trained.
for column in forecast_model.X_train.columns:
    if column not in processed_future_data.columns:
        processed_future_data[column] = 0  # Añadir columnas faltantes con valores 0

# Rearrange the columns to match the order of X_train
processed_future_data = processed_future_data[forecast_model.X_train.columns]

# Using the model to make predictions
predicted_amounts = forecast_model.predict(processed_future_data)

# Show predictions
print(f"\nDate: 2023-06-21, Amount Forecasted: {predicted_amounts[0]:.4f}")


Extract date features
Handle missing values
Normalize 'ORIGINAL GROSS AMT' column
Encode categorical features using Label Encoding
Group less frequent categories into a new category called 'Others' for TRANS TAX DESC
Group less frequent categories into a new category called 'Others' for ORIGINAL CUR
Group less frequent categories into a new category called 'Others' for DIRECTORATE
Encode categorical features using Label Encoding
Drop columns
Remove duplicate rows

Date: 2023-06-21, Amount Forecasted: 0.0093


2. Expense forecast:

For expense forecasting, you can extend the above method to predict transaction amounts for a range of dates in the future. For example, if you want to forecast expenses for the next month:

In [18]:
start_date = transactions_data['TRANS DATE'].max() + pd.Timedelta(days=1)
end_date = start_date + pd.Timedelta(days=30)
date_range = pd.date_range(start_date, end_date)
date_str_list = [date.strftime('%Y-%m-%d') for date in date_range]

length = len(date_str_list)

trans_tax_dec_values = transactions_data['TRANS TAX DESC'].unique().tolist()
trans_tax_dec_values = [random.choice(trans_tax_dec_values) for _ in range(length)]

original_gross_amt = [random.uniform(-50, 2500) for _ in range(length)]

original_cur_values = transactions_data['ORIGINAL CUR'].unique().tolist()
original_cur_values = [random.choice(original_cur_values) for _ in range(length)]

merchant_name_values = transactions_data['MERCHANT NAME'].unique().tolist()
merchant_name_values = [random.choice(merchant_name_values) for _ in range(length)]

directorate_values = transactions_data['DIRECTORATE'].unique().tolist()
directorate_values = [random.choice(directorate_values) for _ in range(length)]

future_data = pd.DataFrame({
    'TRANS DATE': date_str_list,
    'TRANS TAX DESC': trans_tax_dec_values,
    'ORIGINAL GROSS AMT': original_gross_amt,
    'ORIGINAL CUR': original_cur_values,
    'BILLING CUR CODE': original_cur_values,
    'BILLING GROSS AMT': original_gross_amt,
    'MERCHANT NAME': merchant_name_values,
    'CARD NUMBER': ["*****1234"]*length,
    'TRANS CAC CODE 1': ["L100"]*length,
    'TRANS CAC CODE 2': ["REAAB"]*length,
    'DIRECTORATE': directorate_values,
})

# Usar DataProcessor para procesar los datos futuros
data_processor = DataProcessor(future_data)
processed_future_data = data_processor.get_processed_data(
    column_to_normalize="ORIGINAL GROSS AMT",
    categorical_columns=["MERCHANT NAME"],
    immbalance_colummns=["TRANS TAX DESC", "ORIGINAL CUR", "DIRECTORATE"],
    columns_to_drop=["TRANS DATE", "BILLING CUR CODE", "BILLING GROSS AMT", "CARD NUMBER", "TRANS CAC CODE 1", "TRANS CAC CODE 2"]
)

# Let's make sure that the processed_future_data columns match the training columns.
# This is crucial because the model expects to receive the same features with which it was trained.
for column in forecast_model.X_train.columns:
    if column not in processed_future_data.columns:
        processed_future_data[column] = 0  # Añadir columnas faltantes con valores 0

# Rearrange the columns to match the order of X_train
processed_future_data = processed_future_data[forecast_model.X_train.columns]

# Using the model to make predictions
predicted_amounts = forecast_model.predict(processed_future_data)
total_forecasted_expenditure = sum(predicted_amounts)

# Show predictions
print(f"\nThe total expected expenditure from '2022-07-02' to '2022-08-01' is {total_forecasted_expenditure:.4f}")


Extract date features
Handle missing values
Normalize 'ORIGINAL GROSS AMT' column
Encode categorical features using Label Encoding
Group less frequent categories into a new category called 'Others' for TRANS TAX DESC
Group less frequent categories into a new category called 'Others' for ORIGINAL CUR
Group less frequent categories into a new category called 'Others' for DIRECTORATE
Encode categorical features using Label Encoding
Drop columns
Remove duplicate rows

The total expected expenditure from '2022-07-02' to '2022-08-01' is 0.4410
